In [1]:
import torch

/home/nayakab/Desktop/python-virtual-env/env/lib/python3.6/site-packages/torch/package/_mock_zipreader.py:17: UserWarning: Failed to initialize NumPy: numpy.core.multiarray failed to import (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:67.)
  _dtype_to_storage = {data_type(0).dtype: data_type for data_type in _storages}


In [37]:
x = torch.randn(3, requires_grad=True)
print(x)

tensor([ 1.4504,  0.1775, -0.4907], requires_grad=True)


In [38]:
y = x ** 2
y.retain_grad()
y

tensor([2.1036, 0.0315, 0.2408], grad_fn=<PowBackward0>)

In [39]:
z = y ** 2 + 2 
z

tensor([6.4252, 2.0010, 2.0580], grad_fn=<AddBackward0>)

In [40]:
# Jacobian Vector product - specify the vector or use a scalar
#z.sum().backward()
vector_v = torch.Tensor(3)
z.backward(vector_v)

In [41]:
x.grad

tensor([-2.4805e+13,  1.0258e-42, -2.3197e-36])

In [42]:
y.grad

tensor([-8.5512e+12,  2.8881e-42,  2.3635e-36])

In [44]:
# Blocking gradients
x.requires_grad_(False)
x.detach()
x.requires_grad_(True)
with torch.no_grad():
    # No gradient for y because no gradient for x
    y = x ** 2
    print(y)    

tensor([2.1036, 0.0315, 0.2408])


In [46]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(2):
    output = weights * 3 
    output = output.sum()
    
    output.backward()
    
    print(weights.grad)
    
    # Make Grad zero everytime because Torch keeps accumulating these gradients  
    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


### Backpropagation

In [1]:
# Computational Graph is created with each calculation
import torch 
x = torch.tensor(1.0)
y = torch.tensor(2.0)

weights = torch.tensor(1.0, requires_grad=True)

# Forward
y_ = weights * x
loss = (y_ - y) ** 2
print(loss)

loss.backward() 
print(weights.grad)

tensor(1., grad_fn=<PowBackward0>)
tensor(-2.)


/home/nayakab/Desktop/python-virtual-env/env/lib/python3.6/site-packages/torch/package/_mock_zipreader.py:17: UserWarning: Failed to initialize NumPy: numpy.core.multiarray failed to import (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:67.)
  _dtype_to_storage = {data_type(0).dtype: data_type for data_type in _storages}


In [4]:
import numpy as np

X = np.array([1, 2, 3, 4], dtype=np.float32)
Y = np.array([2, 4, 6, 8], dtype=np.float32)

w = 0.0

def forward(input, weights):
    return weights * input

def loss(Y, Y_pred):
    return ((Y_pred - Y) ** 2).mean()

def gradient(x, Y, Y_predicted):
    return 2 * np.dot(Y_predicted - Y, x).mean()

learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    Y_pred = forward(X, w)
    l = loss(Y, Y_pred)
    w_grad = gradient(X, Y, Y_pred)
    
    w -= learning_rate * w_grad
    
    if epoch % 1 == 0:
        print("Epoch {} === Weight : {}, Loss : {}".format(epoch, w, l))
 

Epoch 0 === Weight : 1.2, Loss : 30.0
Epoch 1 === Weight : 1.6799999618530272, Loss : 4.799999237060547
Epoch 2 === Weight : 1.871999988555908, Loss : 0.7680001854896545
Epoch 3 === Weight : 1.9487999868392942, Loss : 0.1228799968957901
Epoch 4 === Weight : 1.9795200133323667, Loss : 0.019660834223031998
Epoch 5 === Weight : 1.9918080282211301, Loss : 0.0031457357108592987
Epoch 6 === Weight : 1.9967231869697568, Loss : 0.0005033080233260989
Epoch 7 === Weight : 1.99868928194046, Loss : 8.053186320466921e-05
Epoch 8 === Weight : 1.999475698471069, Loss : 1.2884394891443662e-05
Epoch 9 === Weight : 1.999790253639221, Loss : 2.0613531432900345e-06
Epoch 10 === Weight : 1.9999160599708554, Loss : 3.297340072094812e-07
Epoch 11 === Weight : 1.9999664139747617, Loss : 5.282345227897167e-08
Epoch 12 === Weight : 1.9999865984916685, Loss : 8.487816671731707e-09
Epoch 13 === Weight : 1.9999946093559262, Loss : 1.3369572116062045e-09
Epoch 14 === Weight : 1.9999978351593015, Loss : 2.1679014139

In [13]:
# Now with Torch
import torch

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

def forward(x, w):
    return x * w

def loss(Y, Y_pred):
    return ((Y_pred - Y) ** 2).mean()

learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    Y_pred = forward(X, w)
    
    l = loss(Y, Y_pred)
    
    l.backward()
    
    with torch.no_grad():
        w -= learning_rate * w.grad
        
    if epoch % 10 == 0:
        print("Epoch {} === Weight : {}, Loss : {}".format(epoch, w, l))
        
    w.grad.zero_()

Epoch 0 === Weight : 0.29999998211860657, Loss : 30.0
Epoch 10 === Weight : 1.6653136014938354, Loss : 1.1627856492996216
Epoch 20 === Weight : 1.934108853340149, Loss : 0.0450688973069191
Epoch 30 === Weight : 1.987027645111084, Loss : 0.0017468547448515892
Epoch 40 === Weight : 1.9974461793899536, Loss : 6.770494655938819e-05
Epoch 50 === Weight : 1.9994971752166748, Loss : 2.6243997126584873e-06
Epoch 60 === Weight : 1.9999010562896729, Loss : 1.0175587306093803e-07
Epoch 70 === Weight : 1.9999804496765137, Loss : 3.9741685498029256e-09
Epoch 80 === Weight : 1.999996304512024, Loss : 1.4670220593870908e-10
Epoch 90 === Weight : 1.9999992847442627, Loss : 5.076827847005916e-12
